<a href="https://colab.research.google.com/github/sahithikodali1/Summarization-of-Biomedical-evidence/blob/Data_processing/BIOASQ_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os

print(os.listdir("/content/drive/MyDrive/Thesis_B"))

['rouge_8b_mod.csv', '.ipynb_checkpoints', 'test_data.csv', 'train_data.csv']


In [2]:
DATA_DIR = "/content/drive/MyDrive/Thesis_B"

file = pd.read_csv(os.path.join(DATA_DIR,'rouge_8b_mod.csv'))
f_values = file.values
# print(f_values[:10])
# print(col_names[:10])
q_id_split = np.split(f_values, np.where(np.diff(f_values[:,0]))[0]+1)
# print(q_id_split[:2])

In [3]:
train_data_size = round(len(q_id_split)*0.8)
test_data_size = round(len(q_id_split) - train_data_size)
print(train_data_size)
print(test_data_size)

2594
648


In [4]:
import random
random.seed(3007)
random.shuffle(q_id_split)
print(type(q_id_split[:1]))

<class 'list'>


In [7]:
train_data_beforeval = q_id_split[:2594]
f_values_train_beforeval = np.concatenate(train_data_beforeval, axis=0)
# print((f_values_train_beforeval))

q_id_split_train = np.split(f_values_train_beforeval, np.where(np.diff(f_values_train_beforeval[:,0]))[0]+1)
print(len(q_id_split_train))

2594


In [8]:
val_data_size = round(len(q_id_split_train)*0.2)
train_data_size = round(len(q_id_split_train) - val_data_size)
print(train_data_size)
print(val_data_size)

2075
519


In [9]:
train_data = q_id_split_train[:2075]
val_data = q_id_split_train[2075:]
test_data = q_id_split[2594:]
f_values_train = np.concatenate(train_data, axis=0)
f_values_val = np.concatenate(val_data, axis=0)
f_values_test = np.concatenate(test_data, axis=0)
col_names = file.columns
print(col_names)

train_df = pd.DataFrame(f_values_train, columns = col_names)
val_df = pd.DataFrame(f_values_val, columns = col_names)
test_df = pd.DataFrame(f_values_test, columns = col_names)

Index(['qid', 'pubmedid', 'sentid', 'N1', 'N2', 'L', 'S4', 'SU4',
       'sentence text', 'SU4_labels'],
      dtype='object')


In [10]:
print(len(train_data))
print(len(val_data))
print(len(test_data))

print(len(train_df))
print(len(val_df))
print(len(test_df))
print(train_df.columns.values)

2075
519
648
33435
8148
10161
['qid' 'pubmedid' 'sentid' 'N1' 'N2' 'L' 'S4' 'SU4' 'sentence text'
 'SU4_labels']


In [11]:
# install
!pip install pytorch-pretrained-bert pytorch-nlp

# BERT imports
import torch

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange

import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [13]:
def data_grouplists(data):
  data_list = data.groupby('qid', as_index=False)['SU4','SU4_labels','sentence text'].agg(list)
  return data_list

In [14]:
train_list = data_grouplists(train_df)
val_list = data_grouplists(val_df)
test_list = data_grouplists(test_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [15]:
def add_specialtokens(data_list):
  specialtok_sentences = []

  for sentence in data_list['sentence text']:
    each_specialtoken = []
    for each in sentence:
      each_specialtoken.append(["[CLS] " + each + " [SEP]" ])
    specialtok_sentences.append(each_specialtoken)
  return specialtok_sentences

In [16]:
train_specialtok = add_specialtokens(train_list)
val_specialtok = add_specialtokens(val_list)
test_specialtok = add_specialtokens(test_list)

In [17]:
print(train_specialtok[0])
print(train_specialtok[1])

[['[CLS] Papilin is an extracellular matrix glycoprotein [SEP]'], ['[CLS] apilins are homologous, secreted extracellular matrix proteins which share a common order of protein domains. [SEP]'], ['[CLS] Both MIG-6 isoforms have a predicted N-terminal papilin cassette [SEP]'], ['[CLS] The TSR superfamily is a diverse family of extracellular matrix and transmembrane proteins, many of which have functions related to regulating matrix organization, cell-cell interactions and cell guidance. [SEP]'], ['[CLS]  Collagen IV, laminin, glutactin, papilin, and other extracellular matrix proteins were made primarily by hemocytes and were secreted into the medium. [SEP]'], ['[CLS] A sulfated glycoprotein was isolated from the culture media of Drosophila Kc cells and named papilin. [SEP]'], ['[CLS] Using expression analysis, we identify three genes that are transcriptionally regulated by HLH-2: the protocadherin cdh-3, and two genes encoding secreted extracellular matrix proteins, mig-6/papilin and him

In [18]:
def tokenize_sentences(specialtok_sentences):
  tokenized_sentences = []

  for sentence in specialtok_sentences:
    eachsent_tokenized = []
    for each in sentence:
      eachsent_tokenized.append(tokenizer.tokenize(each[0]))
    tokenized_sentences.append(eachsent_tokenized)
  return tokenized_sentences

In [19]:
train_tokenized = tokenize_sentences(train_specialtok)
val_tokenized = tokenize_sentences(val_specialtok)
test_tokenized = tokenize_sentences(test_specialtok)


In [20]:
def data_tokenizedsent(tokenized_sentences, data_list):
  data_list['tokenized_sentences'] = tokenized_sentences
  return data_list

In [21]:
print(train_tokenized[0])

[['[CLS]', 'pa', '##pi', '##lin', 'is', 'an', 'extra', '##cellular', 'matrix', 'g', '##ly', '##co', '##pro', '##tein', '[SEP]'], ['[CLS]', 'api', '##lins', 'are', 'homo', '##log', '##ous', ',', 'secret', '##ed', 'extra', '##cellular', 'matrix', 'proteins', 'which', 'share', 'a', 'common', 'order', 'of', 'protein', 'domains', '.', '[SEP]'], ['[CLS]', 'both', 'mig', '-', '6', 'iso', '##forms', 'have', 'a', 'predicted', 'n', '-', 'terminal', 'pa', '##pi', '##lin', 'cassette', '[SEP]'], ['[CLS]', 'the', 'ts', '##r', 'superfamily', 'is', 'a', 'diverse', 'family', 'of', 'extra', '##cellular', 'matrix', 'and', 'trans', '##me', '##mb', '##rane', 'proteins', ',', 'many', 'of', 'which', 'have', 'functions', 'related', 'to', 'regulating', 'matrix', 'organization', ',', 'cell', '-', 'cell', 'interactions', 'and', 'cell', 'guidance', '.', '[SEP]'], ['[CLS]', 'col', '##lage', '##n', 'iv', ',', 'lam', '##ini', '##n', ',', 'g', '##lu', '##ta', '##ct', '##in', ',', 'pa', '##pi', '##lin', ',', 'and', 'o

In [22]:
data_train_tokenized = data_tokenizedsent(train_tokenized, train_list)
data_val_tokenized = data_tokenizedsent(val_tokenized, val_list)
data_test_tokenized = data_tokenizedsent(test_tokenized, test_list)

In [23]:
print(data_train_tokenized.columns.values)

['qid' 'SU4' 'SU4_labels' 'sentence text' 'tokenized_sentences']


In [24]:
print(len(train_tokenized[0]))
print(len(train_tokenized[1]))
print(len(train_tokenized[2]))

11
13
32


In [25]:
def token2ids(tokenized_sentences):
  # Set the maximum sequence length. 
  MAX_LEN = 512
  token_ids = []

  for tokenized_sentence in tokenized_sentences:
    tokens_to_ids = [tokenizer.convert_tokens_to_ids(each) for each in tokenized_sentence]
    tokens_to_ids = pad_sequences(tokens_to_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    token_ids.append(tokens_to_ids)
  return token_ids

In [26]:
train_tokens2ids = token2ids(train_tokenized)
val_tokens2ids = token2ids(val_tokenized)
test_tokens2ids = token2ids(test_tokenized)

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (529 > 512). Running this sequence through BERT will result in indexing errors


In [27]:
print(len(train_tokens2ids))
print(len(train_tokens2ids[0]))
print(len(train_tokens2ids[1]))

2075
11
13


In [28]:
def create_masks(token_ids):
  attention_masks = []

  for tid in token_ids:
    eachtid_mask = []
    for each in tid:
      mask = [float(i>0) for i in each] #for each token 
      eachtid_mask.append(mask)#for each sentence
    attention_masks.append(eachtid_mask)
  return attention_masks

In [29]:
train_masks = create_masks(train_tokens2ids)
val_masks = create_masks(val_tokens2ids)
test_masks = create_masks(test_tokens2ids)

In [30]:
print(len(train_masks))
print(len(train_masks[0]))
print(len(train_masks[0][0]))
print(len(train_masks[1]))
print(len(train_masks[1][1]))

print(len(val_masks))

print(len(test_masks))


2075
11
512
13
512
519
648


In [ ]:
#convert data into tensors

In [ ]:
#define the model

In [ ]:
#train the model